In [32]:
import warnings

import numpy as np
import pandas as pd

from itertools import product
from perlin_numpy import generate_perlin_noise_2d
from scipy.interpolate import griddata
from scipy.stats import percentileofscore, norm
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import RBF
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures

from scipy.optimize import minimize



In [33]:
# Random seed
np.random.seed(1)

m = 5 # Physical observations per iteration (Default 5)
n = 100 # Simulated observations per iteration (Default 100)

# Number of observations to train the GP on before starting the active learning loop
pretrain_n = 1

# Minimum percentile to explore each iteration
percentile = 50

# Number of iterations to run the active learning loop (Default 10)
iterations = 10

# Corrective constant when calculating percentages
laplace_alpha = 0.01

# Degree of the polynomial to fit to the data
degree = 2

In [39]:
# List of benchmark functions to test
ackley = lambda X, Y: -(20 * (1 - np.exp(-0.2 * np.sqrt(0.5 * (X**2 + Y**2)))) - np.exp(0.5 * (np.cos(2 * np.pi * X) + np.cos(2 * np. pi * Y))) + np.exp(1))
beale = lambda X, Y: -((1.5 - X + X * Y)**2 + (2.25 - X + X * Y**2)**2 + (2.625 - X + X * Y**3)**2)
bird = lambda X, Y: -(np.sin(X) * np.exp((1 - np.cos(Y))**2) + np.cos(Y) * np.exp((1 - np.sin(X))**2) + (X - Y)**2)
booth = lambda X, Y: -((X + 2 * Y - 7)**2 + (2 * X + Y - 5)**2)
bukin6 = lambda X, Y: -(100 * np.sqrt(abs(Y - 0.01 * X**2)) + 0.01 * np.abs(X + 10))
crossintray = lambda X, Y: 0.0001 * (np.abs(np.sin(X) * np.sin(Y) * np.exp(np.abs(100 - np.sqrt(X**2 + Y**2) / np.pi))) + 1)**0.1
crownedcross = lambda X, Y: -0.0001 * (np.abs(np.sin(X) * np.sin(Y) * np.exp(np.abs(100 - np.sqrt(X**2 + Y**2) / np.pi))) + 1)**0.1
goldsteinprice = lambda X, Y: -(1 + (X + Y + 1)**2 * (19 - 14 * X + 3 * X**2 - 14 * Y + 6 * X * Y + 3 * Y**2)) * (30 + (2 * X - 3 * Y)**2 * (18 - 32 * X + 12 * X**2 + 48 * Y - 36 * X * Y + 27 * Y**2))
griewank = lambda X, Y: -((X**2 + Y**2) / 200 - np.cos(X) * np.cos(Y / np.sqrt(2)) + 1)
himmelblau = lambda X, Y: -((X**2 + Y - 11)**2 + (X + Y**2 - 7)**2)
holdertable = lambda X, Y: np.abs(np.sin(X) * np.cos(Y) * np.exp(np.abs(1 - np.sqrt(X**2 + Y**2) / np.pi)))
leon = lambda X, Y: -(100 * (Y - X**3)**2 + (1 - X)**2)
levi13 = lambda X, Y: -(np.sin(3 * np.pi * X)**2 + (X - 1)**2 * (1 + np.sin(3 * np.pi * Y)**2) + (Y - 1)**2 * (1 + np.sin(2 * np.pi * Y)**2))
matyas = lambda X, Y: -(0.26 * (X**2 + Y**2) - 0.48 * X * Y)
mccormick = lambda X, Y: -(np.sin(X + Y) + (X - Y)**2 - 1.5 * X + 2.5 * Y + 1)
penholder = lambda X, Y: np.exp(-(np.abs(np.cos(X) * np.cos(Y) * np.exp(np.abs(1 - np.sqrt(X**2 + Y**2) / np.pi))))**-1)
perlin = lambda X, Y: generate_perlin_noise_2d((X.shape[0], X.shape[1]), (2, 2))
rastrigin = lambda X, Y: -(X**2 + Y**2 - 10 * np.cos(2 * np.pi * X) - 10 * np.cos(2 * np.pi * Y) + 2)
schweffel = lambda X, Y: X * np.sin(np.sqrt(np.abs(X))) + Y * np.sin(np.sqrt(np.abs(Y)))
sixhumpcamel = lambda X, Y: -((4 - 2.1 * X**2 + X**4 / 3) * X**2 + X * Y + (4 * Y**2 - 4) * Y**2)
testtubeholder = lambda X, Y: 4 * np.abs(np.sin(X) * np.cos(Y) * np.exp(np.abs(np.cos((X**2 + Y**2) / 200))))
zettl = lambda X, Y: -((X**2 + Y**2 - 2 * X)**2 + X / 4)

# Dictionary of benchmark functions with their bounds
objectives = {
    "ackley": {
        "func": ackley,
        "bounds": [(-35, 35)] * 2
    },
    "beale": {
        "func": beale,
        "bounds": [(-4.5, 4.5)] * 2
    },
    "bird": {
        "func": bird,
        "bounds": [(-2 * np.pi, 2 * np.pi)] * 2
    },
    "booth": {
        "func": booth,
        "bounds": [(-10, 10)] * 2
    },
    "bukin6": {
        "func": bukin6,
        "bounds": [(-15, 5), (-3, 3)]
    },
    "crossintray": {
        "func": crossintray,
        "bounds": [(-10, 10)] * 2
    },
    "crownedcross": {
        "func": crownedcross,
        "bounds": [(-10, 10)] * 2
    },
    "goldsteinprice": {
        "func": goldsteinprice,
        "bounds": [(-2, 2)] * 2
    },
    "griewank": {
        "func": griewank,
        "bounds": [(-100, 100)] * 2
    },
    "himmelblau": {
        "func": himmelblau,
        "bounds": [(-5, 5)] * 2
    },
    "holdertable": {
        "func": holdertable,
        "bounds": [(-10, 10)] * 2
    },
    "leon": {
        "func": leon,
        "bounds": [(-1.2, 1.2)] * 2
    },
    "levi13": {
        "func": levi13,
        "bounds": [(-10, 10)] * 2
    },
    "matyas": {
        "func": matyas,
        "bounds": [(-10, 10)] * 2
    },
    "mccormick": {
        "func": mccormick,
        "bounds": [(-1.5, 4), (-3, 4)]
    },
    "penholder": {
        "func": penholder,
        "bounds": [(-11, 11)] * 2
    },
    # "perlin": {
    #     "func": perlin,
    #     "bounds": [(-1, 1)] * 2
    # },
    "rastrigin": {
        "func": rastrigin,
        "bounds": [(-5.12, 5.12)] * 2
    },
    "schweffel": {
        "func": schweffel,
        "bounds": [(-500, 500)] * 2
    },
    "sixhumpcamel": {
        "func": sixhumpcamel,
        "bounds": [(-5, 5)] * 2
    },
    "testtubeholder": {
        "func": testtubeholder,
        "bounds": [(-10, 10)] * 2
    },
    "zettl": {
        "func": zettl,
        "bounds": [(-5, 5)] * 2
    }
}

In [40]:
ran_err = lambda n, x: np.random.normal(0, x, n)
sys_err = lambda n, x, y, poly_model: poly_model.intercept_ + poly_model.coef_[0] * x + poly_model.coef_[1] * y + poly_model.coef_[2] * x**2 + poly_model.coef_[3] * y**2 + poly_model.coef_[4] * x * y

In [41]:
def expected_improvement(x, model, y_max, xi=0.01):
    mu, sigma = model.predict(x.reshape(1, -1), return_std=True)
    with np.errstate(divide='warn'):
        imp = mu - y_max - xi
        Z = imp / sigma
        ei = imp * norm.cdf(Z) + sigma * norm.pdf(Z)
        ei[sigma == 0.0] = 0.0
    return -ei

In [42]:
# Silence repetitive warnings
warnings.simplefilter('ignore')

# Active learning loop
for obj in list(objectives.keys()):
    # Get the bounds for the function
    xy_range = objectives[obj]["bounds"]

    # Generate the meshgrid for the function
    X = np.arange(*xy_range[0], ((xy_range[0][1] - xy_range[0][0]) / 100))
    Y = np.arange(*xy_range[1], ((xy_range[1][1] - xy_range[1][0]) / 100))
    X, Y = np.meshgrid(X, Y)
    Z = objectives[obj]["func"](X, Y)

    # Get the range of the function
    z_range = (np.floor(np.min(Z)) - 1, np.ceil(np.max(Z)) + 1)

    # Generate the initial training data
    df = pd.DataFrame(np.random.randint(100, size=(pretrain_n, 2)))

    # Rename the index columns
    df.columns = ["i", "j"]

    # Calculate the x, y, and z columns with some random error
    df["x"] = X[0, df["i"]]
    df["y"] = Y[df["j"], 0]
    df["z"] = Z[df["i"], df["j"]] + ran_err(pretrain_n, 0.05)

    # Create a copy of the dataframe to store the simulated data
    simul_df = df.copy()

    # Fit a polynomial to the data
    poly_features = PolynomialFeatures(degree=degree, include_bias=False)
    poly_features = poly_features.fit_transform(simul_df[["x", "y"]])

    # Fit a linear regression model to polynomial
    poly_model = LinearRegression()
    poly_model = poly_model.fit(poly_features, simul_df["z"])

    # Generate some more initial training data
    df = pd.DataFrame(np.random.randint(100, size=(m, 2)))

    # Rename the index columns
    df.columns = ["i", "j"]

    # Calculate the x, y, and z columns with some random error
    df["x"] = X[0, df["i"]]
    df["y"] = Y[df["j"], 0]
    df["z"] = Z[df["i"], df["j"]] + ran_err(m, 0.05)

    for idx in range(1, iterations + 1):
        #Fit a GP to the data
        kernel = RBF(length_scale=1)
        model = GaussianProcessRegressor(kernel=kernel, normalize_y=False, random_state=3, alpha=0.001)

        # Randomly sample within our function's bounds
        tmp_df = pd.DataFrame(np.random.randint(100, size=(n, 2)))

        # Rename the index columns
        tmp_df.columns = ["i", "j"]

        # Generate some simulated data
        tmp_df["x"] = X[0, tmp_df["i"]]
        tmp_df["y"] = Y[tmp_df["j"], 0]
        tmp_df["z"] = sys_err(n, tmp_df["x"], tmp_df["y"], poly_model)

        # Fit a GP to the data
        model.fit(tmp_df[["x", "y"]], tmp_df["z"])

        # Make predictions using simulated data
        pred = model.predict(tmp_df[["x", "y"]])
        y_max = pred.max()

        # Find the next sampling point using the acquisition function
        res = minimize(fun=expected_improvement, x0=np.random.uniform(xy_range[0][0], xy_range[0][1], size=(1, 2)), 
                       bounds=xy_range, args=(model, y_max))
        next_point = res.x

        # Sample new points based on the next point
        tmp_df = pd.DataFrame({'x': [next_point[0]], 'y': [next_point[1]], 
                               'z': [objectives[obj]['func'](next_point[0], next_point[1]) + np.random.normal(0, 0.05)]})

        df = pd.concat([df, tmp_df], ignore_index=True)

        simul_df = pd.concat([simul_df, tmp_df], ignore_index=True)

        poly_features = PolynomialFeatures(degree=degree, include_bias=False)
        poly_features = poly_features.fit_transform(simul_df[["x", "y"]])

        poly_model = LinearRegression()
        poly_model = poly_model.fit(poly_features, simul_df["z"])

    krnl = RBF(length_scale=1)
    model = GaussianProcessRegressor(kernel=krnl, normalize_y=False, random_state=3, alpha=0.001)

    model.fit(df[["x", "y"]], df["z"])

    pred = model.predict(df[["x", "y"]])

    z = griddata((df["x"], df["y"]), pred, (X.T, Y.T), method="linear", fill_value=(z_range[0] + 1))

    print(obj, percentileofscore(Z.flatten(), Z[np.unravel_index(z.argmax(), z.shape)]), sep="\t", end="\r\n")

ackley	99.305
beale	76.74
bird	69.21
booth	99.49
bukin6	93.89
crossintray	98.475
crownedcross	97.165
goldsteinprice	85.08
griewank	89.115
himmelblau	81.62
holdertable	69.61
leon	85.13
levi13	89.77
matyas	97.605
mccormick	97.32
penholder	95.775
rastrigin	90.685
schweffel	98.205
sixhumpcamel	91.88
testtubeholder	28.09
zettl	94.91
